# Open AI modelių pritaikymas (Fine Tuning)

Šis užrašų sąsiuvinis remiasi dabartinėmis [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentacijos gairėmis iš Open AI.

Modelio pritaikymas pagerina bazinių modelių veikimą jūsų programoje, nes modelis papildomai apmokomas su papildomais duomenimis ir kontekstu, kurie yra svarbūs konkrečiam naudojimo atvejui ar scenarijui. Atkreipkite dėmesį, kad promptų kūrimo metodai, tokie kaip _few shot learning_ ir _retrieval augmented generation_, leidžia pagerinti numatytąjį promptą su aktualiais duomenimis, kad pagerėtų kokybė. Tačiau šie metodai yra ribojami pasirinkto bazinio modelio maksimalaus žetonų lango dydžio.

Naudodami pritaikymą, mes iš esmės iš naujo apmokome patį modelį su reikiamais duomenimis (leidžiame naudoti daug daugiau pavyzdžių nei telpa į maksimalaus žetonų lango ribas) ir diegiame _individualią_ modelio versiją, kuriai jau nebereikia pateikti pavyzdžių prognozavimo metu. Tai ne tik pagerina mūsų promptų kūrimo efektyvumą (turime daugiau lankstumo, kaip išnaudoti žetonų langą kitiems dalykams), bet ir gali sumažinti mūsų kaštus (nes sumažėja žetonų, kuriuos reikia siųsti modeliui prognozavimo metu, kiekis).

Modelio pritaikymo procesas susideda iš 4 žingsnių:
1. Paruoškite mokymo duomenis ir juos įkelkite.
1. Paleiskite mokymo užduotį, kad gautumėte pritaikytą modelį.
1. Įvertinkite pritaikytą modelį ir, jei reikia, tobulinkite kokybę.
1. Kai būsite patenkinti, diekite pritaikytą modelį prognozavimui.

Atkreipkite dėmesį, kad ne visi baziniai modeliai palaiko pritaikymą – [patikrinkite OpenAI dokumentaciją](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) dėl naujausios informacijos. Taip pat galite pritaikyti jau anksčiau pritaikytą modelį. Šioje pamokoje kaip bazinį modelį pritaikymui naudosime `gpt-35-turbo`. 

---


### 1.1 žingsnis: Paruoškite savo duomenų rinkinį

Kurkime pokalbių robotą, kuris padės suprasti periodinę elementų lentelę, atsakydamas į klausimus apie elementą limeriko forma. Šiame paprastame vadove tiesiog sukursime duomenų rinkinį, kad apmokytume modelį su keliomis pavyzdinėmis atsakymų eilutėmis, kurios parodo, kokio formato duomenų tikimasi. Tikrame projekte jums reikėtų daug daugiau pavyzdžių. Taip pat galite pasinaudoti atviru duomenų rinkiniu (jei toks yra jūsų taikymo srityje) ir jį pritaikyti smulkiam modelio derinimui.

Kadangi orientuojamės į `gpt-35-turbo` ir siekiame vieno atsakymo (chat completion), galime kurti pavyzdžius pagal [rekomenduojamą formatą](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), atitinkantį OpenAI pokalbių užbaigimo reikalavimus. Jei tikitės kelių žinučių pokalbio, naudotumėte [kelių žinučių pavyzdžių formatą](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), kuriame yra `weight` parametras, nurodantis, kurios žinutės turi būti naudojamos (ar ne) smulkiam derinimui.

Šiame vadove naudosime paprastesnį vieno atsakymo formatą. Duomenys pateikiami [jsonl formatu](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), kur kiekviena eilutė yra atskiras JSON objektas. Žemiau pateiktame fragmente matote 2 pavyzdžius – visą pavyzdžių rinkinį (10 pavyzdžių), kurį naudosime smulkaus derinimo vadove, rasite [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) faile. **Pastaba:** Kiekvienas įrašas _privalo_ būti vienoje eilutėje (ne išskaidytas per kelias eilutes, kaip dažnai būna formatuotame JSON faile)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Tikrame projekte jums reikės daug didesnio pavyzdžių rinkinio, kad gautumėte gerus rezultatus – teks rinktis tarp atsakymų kokybės ir smulkaus derinimo laiko/kainos. Mes naudojame mažą rinkinį, kad galėtume greitai parodyti smulkaus derinimo procesą. Daugiau apie sudėtingesnį smulkaus derinimo procesą rasite [OpenAI Cookbook pavyzdyje](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst).


### 1.2 žingsnis. Įkelkite savo duomenų rinkinį

Įkelkite duomenis naudodami Files API [kaip aprašyta čia](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Atkreipkite dėmesį, kad norėdami paleisti šį kodą, pirmiausia turite atlikti šiuos veiksmus:
 - Įdiegti `openai` Python paketą (įsitikinkite, kad naudojate versiją >=0.28.0, kad gautumėte naujausias funkcijas)
 - Nustatyti `OPENAI_API_KEY` aplinkos kintamąjį su savo OpenAI API raktu
Daugiau informacijos rasite kurso [paruošimo gide](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst).

Dabar paleiskite kodą, kad sukurtumėte failą įkėlimui iš savo vietinio JSONL failo.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### 2.1 žingsnis: Sukurkite modelio tobulinimo užduotį naudodami SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### 2.2 žingsnis: Patikrinkite darbo būseną

Štai ką galite atlikti naudodami `client.fine_tuning.jobs` API:
- `client.fine_tuning.jobs.list(limit=<n>)` – Išvardija paskutinius n modelio tobulinimo darbus
- `client.fine_tuning.jobs.retrieve(<job_id>)` – Parodo konkretaus modelio tobulinimo darbo informaciją
- `client.fine_tuning.jobs.cancel(<job_id>)` – Atšaukia modelio tobulinimo darbą
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` – Išvardija iki n įvykių iš darbo
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Pirmas šio proceso žingsnis yra _mokymo failo patikrinimas_, kad įsitikintumėte, jog duomenys yra tinkamo formato.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### 2.3 žingsnis: Sekite įvykius, kad stebėtumėte pažangą


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 2.4 žingsnis: Peržiūrėkite būseną OpenAI valdymo skydelyje


Taip pat galite peržiūrėti būseną apsilankę OpenAI svetainėje ir pasirinkę platformos _Fine-tuning_ skiltį. Čia matysite dabartinio darbo būseną ir galėsite sekti ankstesnių darbo vykdymų istoriją. Šioje ekrano nuotraukoje matyti, kad ankstesnis vykdymas nepavyko, o antrasis bandymas buvo sėkmingas. Paaiškinimui: taip nutiko todėl, kad pirmasis bandymas naudojo neteisingai suformatuotą JSON failą – ištaisius klaidą, antrasis bandymas buvo sėkmingai užbaigtas ir modelis tapo prieinamas naudoti.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.lt.png)


Taip pat galite peržiūrėti būsenos pranešimus ir metrikas, slinkdami žemyn vizualioje valdymo panelėje, kaip parodyta:

| Pranešimai | Metrikos |
|:---|:---|
| ![Pranešimai](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.lt.png) |  ![Metrikos](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.lt.png)|


### 3.1 žingsnis: Gaukite ID ir išbandykite patobulintą modelį kode


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### 3.2 žingsnis: Įkelkite ir išbandykite patobulintą modelį Playground aplinkoje

Dabar galite išbandyti patobulintą modelį dviem būdais. Pirmiausia galite apsilankyti Playground aplinkoje ir naudodami Models išskleidžiamąjį meniu pasirinkti savo naujai patobulintą modelį iš pateiktų variantų. Kita galimybė – naudoti „Playground“ parinktį, rodomą Fine-tuning skydelyje (žr. aukščiau pateiktą ekrano nuotrauką), kuri atidaro _palyginamąjį_ vaizdą, kuriame greitai įvertinsite bazinio ir patobulinto modelio atsakymus vienas šalia kito.

Tiesiog įveskite sistemos kontekstą, naudotą jūsų mokymo duomenyse, ir pateikite savo testavimo klausimą. Pastebėsite, kad abiejose pusėse bus atnaujintas tas pats kontekstas ir klausimas. Paleiskite palyginimą ir pamatysite, kuo skiriasi jų atsakymai. _Atkreipkite dėmesį, kaip patobulintas modelis pateikia atsakymą tokiu formatu, kokį nurodėte savo pavyzdžiuose, o bazinis modelis tiesiog seka sistemos nurodymu_.

Taip pat pastebėsite, kad palyginime pateikiamas kiekvieno modelio žetonų skaičius ir laikas, per kurį buvo gautas atsakymas. **Šis konkretus pavyzdys yra labai paprastas ir skirtas parodyti procesą, bet neatspindi realaus pasaulio duomenų ar situacijos**. Galite pastebėti, kad abiejuose pavyzdžiuose žetonų skaičius yra vienodas (sistemos kontekstas ir naudotojo užklausa yra identiški), tačiau patobulintas modelis (individualus modelis) užtrunka ilgiau, kol pateikia atsakymą.

Realiame pasaulyje nenaudosite tokio žaislinio pavyzdžio – modelį derinsite su tikrais duomenimis (pvz., produktų katalogu klientų aptarnavimui), kur atsakymų kokybė bus daug akivaizdesnė. Tokiu atveju, norint pasiekti panašią atsakymo kokybę su baziniu modeliu, reikės daugiau individualaus promptų kūrimo, o tai padidins žetonų naudojimą ir galimai pailgins atsakymo gavimo laiką. _Norėdami tai išbandyti, peržiūrėkite OpenAI Cookbook pateiktus fine-tuning pavyzdžius._



---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo pradinėje kalboje turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojamas profesionalus žmogaus vertimas. Mes neprisiimame atsakomybės už bet kokius nesusipratimus ar neteisingą interpretavimą, kilusį naudojantis šiuo vertimu.
